In [ ]:
# -*- coding: utf-8 -*-
import scrapy
import re 


class StocksSpider(scrapy.Spider):
    name = "stocks"
    #allowed_domains = ["baidu.com"]
    
    #url为初始的url链接
    start_urls = ['http://quote.eastmoney.com/stocklist.html']
    

    #parse函数作用：解析返回的response，从东方财富网
    def parse(self, response):
        #对a标签中的链接进行提取
        for href in response.css('a :: attr(href)').extract():
            
            try:
                #正则表达式获取其中的股票代码
                stock=re.findall(r'[s][hz]\d{6}',href)[0]
                #生成一个对应百度页面的url的链接
                url="https://gupiao.baidu.com/stock/"+stock+'.html'
                #作为一个新的请求，提交给scrapy 框架。self.parse_stock为对应url响应的处理函数，yield将新的链接重新提交给scrapy框架
                yield scrapy.Request(url,callback=self.parse_stock)
            except:
                continue
                
    #从百度的单个页面获取信息的函数，由于函数最终要返回提取的信息给itenpipeline，字典类型
    def parse_stock(self,response):
        infoDict={}
        stockInfo=response.css('.stock-bets')#找到‘。stick-bets的信息’
        #提取相关的字符串extract（）
        name=stockInfo.css('.bets-name').extract()[0]
        keylist=stockInfo.css('dt').extract()
        valuelist=stockInfo.css('dd').extract()
        for i in range(len(keylist)):
            key=re.findall(r'\.*</dt>',keylist[i])[0][1:-5]
            try:
                val=re.findall(r'\d+\.?.*</dd>',valuelist[i])[0][0:-5]
            except:
                val='--'
                infoDict[key]=val
        #infoDict.update(
        #{'股票名称'：re.findall('\s.*\(',name)[0].split()[0]+\re.findall('\>.*\<',name)[0][1:-i]})
        yield infoDict

In [ ]:
# -*- coding: utf-8 -*-

# Define your item pipelines here
#
# Don't forget to add your pipeline to the ITEM_PIPELINES setting
# See: http://doc.scrapy.org/en/latest/topics/item-pipeline.html


class BaidustocksPipeline(object):
    def process_item(self, item, spider):
        return item
class BaidustocksInfoPipeline(object):
    def open_spider(self,spider):
        self.f=open('BaidutockInfo.txt','w')
    def close_spider(self,spider):
        self.f.close()
    def process_item(self,item,spider):
        try:
            line=str(dict(item))+'\n'
            self.f.write(line)
        except:
            pass
        return item
                
        


SyntaxError: invalid syntax (<ipython-input-1-ef0ee20a94e8>, line 1)

In [ ]:
<html>.css